In [ ]:
#@title Mount your Google Drive
%matplotlib inline
%load_ext autoreload
%autoreload 2
import numpy as np
import matplotlib.pyplot as plt
from torchvision.utils import save_image
import torch

from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
#@title Link your assignment folder & install requirements
#@markdown Enter the path to the assignment folder in your Google Drive
import sys
import os
import shutil
import warnings

folder = "/content/gdrive/MyDrive/A3" #@param {type:"string"}
!ln -Ts $folder /content/A3 2> /dev/null

# Add the assignment folder to Python path
if '/content/A3' not in sys.path:
    sys.path.insert(0, '/content/A3')

# Install requirements
!pip install -qr /content/A3/requirements.txt

# Check if CUDA is available
import torch
if not torch.cuda.is_available():
    warnings.warn('CUDA is not available.')

     |████████████████████████████████| 831.4 MB 2.7 kB/s 
     |████████████████████████████████| 22.1 MB 1.4 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchtext 0.11.0 requires torch==1.10.0, but you have torch 1.9.0 which is incompatible.
torchaudio 0.10.0+cu111 requires torch==1.10.0, but you have torch 1.9.0 which is incompatible.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: UserWarning: CUDA is not available.


In [ ]:
%cd /content/gdrive/MyDrive/A3

/content/gdrive/MyDrive/A3


In [ ]:
#!python q2_solution.py
# COMPLETE QUALITATIVE EVALUATION
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model = torch.load("/content/gdrive/MyDrive/A3/G.pkl").cuda()
model.to(device)
    # switch to evaluate mode
model.eval()

try:
    os.makedirs("/content/gdrive/MyDrive/A3")
except OSError:
    pass

In [ ]:
#2.2

from torchvision.utils import save_image
eps = 10
i = 0
z = torch.randn(64, 100, device=device) 
generated = model(z)
save_image(generated.detach(), f"/content/gdrive/MyDrive/A3/generated_{i:04d}.png", normalize=True)

In [ ]:
with torch.no_grad():
  for i in range(100):
    z[:,i] += eps
    fake = model(z)
    save_image(fake.detach(), f"/content/gdrive/MyDrive/A3/fake_{i:04d}.png", normalize=True)
    z[:,i] -= eps

In [ ]:
#2.3 a
# uniform interpolation between two points in latent space
def interpolate_points(p1, p2, n_steps=11):
	# interpolate ratios between the points
	ratios = np.linspace(0, 1, num=n_steps)
	# linear interpolate vectors
	vectors = list()
	for ratio in ratios:
		v = (1.0 - ratio) * p1 + ratio * p2
		vectors.append(v)
	return vectors

In [ ]:
# generate points in latent space
z1 = torch.randn(64, 100, device=device).cpu()
z2 = torch.randn(64, 100, device=device).cpu()  
# interpolate points in latent space
interpolated = interpolate_points(z1, z2)
noises = torch.stack(interpolated, dim=0).to(device)

In [ ]:
with torch.no_grad():
  for i in range(11):
    fake = model(noises[i])
    save_image(fake.detach(), f"/content/gdrive/MyDrive/A3/interpolatedlatent_{i:04d}.png", normalize=True)

In [ ]:
#2.3 b
# uniform interpolation between two points in data space
def interpolate_points(p1, p2, n_steps=11):
	# interpolate ratios between the points
	ratios = np.linspace(0, 1, num=n_steps)
	# linear interpolate vectors
	vectors = list()
	for ratio in ratios:
		v = (1.0 - ratio) * p1 + ratio * p2
		vectors.append(v)
	return vectors

In [ ]:
# generate points in latent space
z1 = torch.randn(64, 100, device=device).to(device)
z2 = torch.randn(64, 100, device=device).to(device)
#predict them by generator
X1 = model(z1)
X2 = model(z2)
X1.shape

torch.Size([64, 3, 32, 32])

In [ ]:
# interpolate points in latent space
interpolatedx = interpolate_points(X1, X2)
noisesx = torch.stack(interpolatedx, dim=0).to(device)
noisesx.shape

torch.Size([11, 64, 3, 32, 32])

In [ ]:
with torch.no_grad():
  for i in range(11):
    save_image(noisesx[i].detach(), f"/content/gdrive/MyDrive/A3/interpolateddata_{i:04d}.png", normalize=True)